In [43]:
import pandas as pd
import numpy as np
import time
import os
import warnings
warnings.filterwarnings('ignore')

stocks_tw_top100 = ['2327','2317', '2330', '2337', '2492', '2344', '2303', '2409', '2891', '3481', '2448', '6153', '2408', '2883', '2888', '2353', '2313', '2371', '2377', '2456', '2002', '3026', '2886', '3037', '2481', '2884', '1101', '2345', '3406', '3231', '2882', '2885', '1605', '6456', '3443', '2892', '2881', '2376', '2887', '2027', '2412', '2049', '2454', '3016', '8163', '3019', '2455', '2367', '2323', '1216', '1314', '2308', '0050', '2498', '2356', '1402', '3673', '4938', '2834', '1102', '2474', '2478', '1301', '5880', '1326', '2439', '3090', '2324', '2603', '3532', '2890', '2382', '6116', '1303', '1312', '2880', '3504', '2375', '2301', '6176', '3035', '3661', '5871', '2610', '4958', '2340', '6120', '3059', '9958', '2014', '2823', '3034', '6269', '9904', '2347', '2105', '2349', '2618', '3596', '0058']
stocks_tw = stocks_tw_top100
stocks_tw = ['2327']

stockDataDir = '/Users/kuen/Desktop/CTA/data/'
cols = ["date","time","lastPx","size","volume","SP5","SP4","SP3","SP2","SP1","BP1","BP2","BP3","BP4","BP5","SV5","SV4","SV3","SV2","SV1","BV1","BV2","BV3","BV4","BV5"]
close= pd.DataFrame(columns =stocks_tw )
size= pd.DataFrame(columns =stocks_tw )
volume=pd.DataFrame(columns =stocks_tw )



In [51]:
# max position 1 lot. signal 0, position doesn't change
class backtest(object):
    def __init__(self,data,signal):

        signal[signal.isna()]=0
        self.data=data
        self.signal=signal
        self.data['signal']=signal
        signal_l = list(self.data.signal)
        position = [data.signal[0]]
        time=list(data.index.strftime("%Y-%m-%d %H:%M"))
        for i in range(1,len(data),1):
            if  ('13:21' in time[i]) or ('13:22' in time[i]) or ('13:23' in time[i]) or \
            ('13:24' in time[i]) or ('13:25' in time[i]):
                position.append(0)
                continue
#             elif signal_l[i]==0: position.append(position[i-1])
            else: position.append(signal_l[i])
        data['position'] = position
        data['position'] = data['position']*1000
        data['trades']=data.position.diff()
        data.trades[0]=data.signal[0]
        data['balance']=(-data.nexttick_px*data.trades-abs(0.00095*data.nexttick_px*data.trades)).cumsum()
        data['value']=data.balance+data.position*data.close
        data.value=data.value.fillna(method='ffill')
        self.data=data
    def wealth(self):
        ret=self.data.value[-1]
        return ret
    def maxdrawdown(self):
        drawdowns = []
        max_so_far = self.data.value[0]
        for i in range(len(self.data.value)):
            if self.data.value[i] > max_so_far:
                drawdown = 0
                drawdowns.append(drawdown)
                max_so_far = self.data.value[i]
            else:
                drawdown = max_so_far - self.data.value[i]
                drawdowns.append(drawdown)
        return max(drawdowns)
    def pnl_day(self):
        pnl=self.data.value.resample('D').last()-self.data.value.resample('D').first()
        return pnl

In [45]:
def clean_data(iStocks):
    ##Iterate through stocks
    print("Processing " + stocks_tw[iStocks])

    ###load stock tick data (gzip)
    file1Path = stockDataDir + stocks_tw[iStocks] + '_md_201801_201812.csv.gz'
    file2Path = stockDataDir + stocks_tw[iStocks] + '_md_201901_201903.csv.gz'
    if os.path.exists(file1Path):
        df = pd.read_csv(file1Path, compression='gzip', usecols = cols)
        print('Data(First file) for ' + stocks_tw[iStocks] + ' loaded.')

        if os.path.exists(file2Path):
            df1 = pd.read_csv(file2Path, compression='gzip', usecols = cols)
            print('Data(Second file) for ' + stocks_tw[iStocks] + ' loaded.')

            df = df.append(df1)
    elif os.path.exists(file2Path):
        df = pd.read_csv(file2Path, compression='gzip', usecols = cols)
        print('Data(Second file) for ' + stocks_tw[iStocks] + ' loaded.')
    else:
        print('Skipping snapshots data for ' + stocks_tw[iStocks] + '.')
#         continue
    df=df[df['SP1']>0]
    df=df[df['BP1']>0]
    df=df[df['SP1']-df['BP1']>0]
    df.index = pd.to_datetime(df['date'], format='%Y-%m-%d') - pd.to_datetime('1900', format='%Y') + pd.to_datetime(df['time'].astype(str).str.slice(0, -5), format='%H%M')
    df['lastPx'] = df['lastPx'] / 100
    df['size'] = df['size'] * 1000
    df['volume'] = df['volume'] * 1000
    data=pd.DataFrame(columns=['nexttick_px','nexttick_size','close','open','high','low','size'],index=df.index)
    data['nexttick_px']=df['lastPx'].shift(-1)
    data['nexttick_size']=df['size'].shift(-1)
    data=data.resample('min').last()
    data['close']=df['lastPx'].resample('min').last()
    data['open']=df['lastPx'].resample('min').first()
    data['high']=df['lastPx'].resample('min').max()
    data['low']=df['lastPx'].resample('min').min()
    data['size']=df['size'].resample('min').sum()
    data[['nexttick_px', 'close', 'open', 'high', 'low']]=data[['nexttick_px', 'close', 'open', 'high', 'low']].fillna(method='ffill')
    data[['size','nexttick_size']]=data[['size','nexttick_size']].fillna(0)
    data.index=pd.to_datetime(data.index)
    data=data.between_time('9:00','13:25')
#     data.to_csv('data_5mins.csv')
    return (data)

In [46]:
def test_all():
    for i in range(len(stocks_tw)):
        dff=clean_data(i)
        for j in range(len(strategy)):
            print(strategy[j])
            fac=eval(strategy[j])(dff.close,dff.open,dff.high,dff.low,dff.size)
            a=backtest(dff,fac)
            ret.iloc[i,j]=a.wealth()
#             print(ret)
            drawdown.iloc[i,j]=a.maxdrawdown()
        print(ret)
    return ret

In [53]:
def bollinger(close,open,high,low,size): 
    window = 200
    s = 1
    r = 8
    t = 0.004
    closedata=pd.concat([close.index.to_series(name='date').astype(str).str.slice(start=0,stop=10),close],axis=1)
    boll = closedata.groupby('date')['close'].rolling(window,min_periods=1).mean()
    sigma = closedata.groupby('date')['close'].rolling(window,min_periods=1).std()
    upper_band = boll + s*(sigma)
    lower_band = boll - s*(sigma)
    close = closedata['close']
    upper_band.index = lower_band.index = boll.index = close.index
    signal= pd.Series({}, index=close.index)
    signal[((close.rolling(r).mean()>=lower_band.rolling(r).mean())&(close<=lower_band*(1-t)))] = (-1)
    signal[((close.rolling(r).mean()>=boll.rolling(r).mean())&(close<=boll*(1-t)))] = (-1)
    signal[((close.rolling(r).mean()<= boll.rolling(r).mean())&(close>=upper_band*(2+t)))] = (1)
#     signal.to_csv('
    data = pd.concat([signal.index.to_series(name='date').astype(str).str.slice(start=0, stop=10), signal], axis=1)
    signal = data.groupby('date')[0].fillna(method='ffill')
    signal = signal.fillna(0)
    signal = signal.reindex(close.index, method='ffill')
    return signal

def PSAR(close,open,high,low,size): 
    window = 30
    r = 8
    t = 0.002
    closedata = pd.concat([close.index.to_series(name='date').astype(str).str.slice(start=0, stop=10), close], axis=1)
    highdata = pd.concat([high.index.to_series(name='date').astype(str).str.slice(start=0, stop=10), high], axis=1)
    lowdata = pd.concat([low.index.to_series(name='date').astype(str).str.slice(start=0, stop=10), low], axis=1)
    trend = (closedata.groupby('date')['close'].diff(1)>0)*1 + (closedata.groupby('date')['close'].diff(1)<0)*(-1)
    LastHigh = highdata.groupby('date')['high'].rolling(window,min_periods=1).max()
    LastLow = lowdata.groupby('date')['low'].rolling(window,min_periods=1).min()
    LastHigh.index = LastLow.index = close.index
    EP = (trend==1)*LastHigh + (trend==(-1))*LastLow
    EP = EP.replace(to_replace=0, method='ffill')
    SAR = [close[0]]
    label = (LastHigh>=LastHigh.cummax())|(LastLow<=LastLow.cummin())
    label = label.astype(float)
    AF = np.minimum(0.0005+np.cumsum(label)*0.0005, 0.2)
    for i in range(1,len(AF)):
        SAR.append(SAR[i-1]+AF[i]*(EP[i]-SAR[i-1]))
    SAR = pd.Series(SAR,index=close.index)
    SAR = SAR.astype(float)
    SAR = SAR.fillna(method='ffill')
    signal= pd.Series({}, index=close.index)
    signal[((SAR.rolling(r).mean()>=close.rolling(r).mean())&(SAR<close*(1-t)))] = (-1) 
    signal[((SAR.rolling(r).mean()<=close.rolling(r).mean())&(SAR>=close*(1+t)))] = (-1)
    data = pd.concat([signal.index.to_series(name='date').astype(str).str.slice(start=0, stop=10), signal], axis=1)
    signal = data.groupby('date')[0].fillna(method='ffill')
    signal = signal.fillna(0)
    signal = signal.reindex(close.index, method='ffill')
    return signal


def williams(close,open,high,low,size): 
    window = 120
    t = 0
    r = 2
    highdata=pd.concat([high.index.to_series(name='date').astype(str).str.slice(start=0,stop=10),high],axis=1)
    lowdata=pd.concat([low.index.to_series(name='date').astype(str).str.slice(start=0,stop=10),low],axis=1)
    HH = highdata.groupby('date').high.rolling(window,min_periods=1).max()
    LL = lowdata.groupby('date').low.rolling(window,min_periods=1).min()
    HH.index= LL.index = close.index
    Will = (-100)*(HH-close)/(HH-LL)
    signal= pd.Series({}, index=close.index)
    signal[((Will.rolling(r).mean()>=(-20*(1-t))) & (Will<(-80)*(1+t)))] = (1)
    signal[((Will.rolling(r).mean()<=(-20*(1+10*t))) & (Will>(-80)*(1-10*t)))] = (-1)
    data = pd.concat([signal.index.to_series(name='date').astype(str).str.slice(start=0, stop=10), signal], axis=1)
    signal = data.groupby('date')[0].fillna(method='ffill')
    signal = signal.fillna(0)
    signal = signal.reindex(close.index, method='ffill')
    return signal

def PROC(close,open,high,low,size): 
    window = 80
    r = 8
    closedata=pd.concat([close.index.to_series(name='date').astype(str).str.slice(start=0,stop=10),close],axis=1)
    PROC = 100*(closedata.groupby('date')['close'].diff(window))/close
    signal= pd.Series({}, index=close.index)
    signal[(((PROC.rolling(r).mean())<= -0.8) & (PROC >  0.8))] = (-1) 
    signal[((PROC<- 0.9))] = (-1)
    data = pd.concat([signal.index.to_series(name='date').astype(str).str.slice(start=0, stop=10), signal], axis=1)
    signal = data.groupby('date')[0].fillna(method='ffill')
    signal = signal.fillna(0)
    signal = signal.reindex(close.index, method='ffill')
    return signal



strategy = ['bollinger']#,'PROC','williams','PSAR']




In [48]:

ret=pd.DataFrame(None,index=stocks_tw,columns=strategy)
drawdown=pd.DataFrame(None,index=stocks_tw,columns=strategy)

start = time.time()
result = test_all()
print(ret.sum())
print((time.time()-start)/60,'mins')



Processing 2327
Data(First file) for 2327 loaded.
Data(Second file) for 2327 loaded.
bollinger
PROC
williams
PSAR
     bollinger    PROC williams    PSAR
2327    689427  246220   698511  732473
2317       NaN     NaN      NaN     NaN
2330       NaN     NaN      NaN     NaN
2337       NaN     NaN      NaN     NaN
2492       NaN     NaN      NaN     NaN
...        ...     ...      ...     ...
2105       NaN     NaN      NaN     NaN
2349       NaN     NaN      NaN     NaN
2618       NaN     NaN      NaN     NaN
3596       NaN     NaN      NaN     NaN
0058       NaN     NaN      NaN     NaN

[100 rows x 4 columns]
Processing 2317
Data(First file) for 2317 loaded.
Data(Second file) for 2317 loaded.
bollinger
PROC
williams
PSAR
     bollinger     PROC williams     PSAR
2327    689427   246220   698511   732473
2317    3765.1 -1753.37 -21624.3 -6576.85
2330       NaN      NaN      NaN      NaN
2337       NaN      NaN      NaN      NaN
2492       NaN      NaN      NaN      NaN
...        ...  

KeyboardInterrupt: 

In [50]:

ret=pd.DataFrame(None,index=stocks_tw,columns=strategy)
drawdown=pd.DataFrame(None,index=stocks_tw,columns=strategy)

start = time.time()
result = test_all()
print(ret.sum())
print((time.time()-start)/60,'mins')




Processing 2327
Data(First file) for 2327 loaded.
Data(Second file) for 2327 loaded.
bollinger
PROC
williams
PSAR
     bollinger    PROC    williams        PSAR
2327    915500  519000  1.0275e+06  1.0465e+06
2317       NaN     NaN         NaN         NaN
2330       NaN     NaN         NaN         NaN
2337       NaN     NaN         NaN         NaN
2492       NaN     NaN         NaN         NaN
...        ...     ...         ...         ...
2105       NaN     NaN         NaN         NaN
2349       NaN     NaN         NaN         NaN
2618       NaN     NaN         NaN         NaN
3596       NaN     NaN         NaN         NaN
0058       NaN     NaN         NaN         NaN

[100 rows x 4 columns]
Processing 2317
Data(First file) for 2317 loaded.
Data(Second file) for 2317 loaded.
bollinger
PROC
williams
PSAR
     bollinger    PROC    williams        PSAR
2327    915500  519000  1.0275e+06  1.0465e+06
2317      4700    9900       26600       11400
2330       NaN     NaN         NaN         

KeyboardInterrupt: 

In [54]:

ret=pd.DataFrame(None,index=stocks_tw,columns=strategy)
drawdown=pd.DataFrame(None,index=stocks_tw,columns=strategy)

start = time.time()
result = test_all()
print(ret.sum())
print((time.time()-start)/60,'mins')





Processing 2327
Data(First file) for 2327 loaded.
Data(Second file) for 2327 loaded.
bollinger
     bollinger
2327    689427
2317       NaN
2330       NaN
2337       NaN
2492       NaN
...        ...
2105       NaN
2349       NaN
2618       NaN
3596       NaN
0058       NaN

[100 rows x 1 columns]
Processing 2317
Data(First file) for 2317 loaded.
Data(Second file) for 2317 loaded.
bollinger
     bollinger
2327    689427
2317    3765.1
2330       NaN
2337       NaN
2492       NaN
...        ...
2105       NaN
2349       NaN
2618       NaN
3596       NaN
0058       NaN

[100 rows x 1 columns]
Processing 2330
Data(First file) for 2330 loaded.
Data(Second file) for 2330 loaded.
bollinger
     bollinger
2327    689427
2317    3765.1
2330  -1653.35
2337       NaN
2492       NaN
...        ...
2105       NaN
2349       NaN
2618       NaN
3596       NaN
0058       NaN

[100 rows x 1 columns]
Processing 2337
Data(First file) for 2337 loaded.
Data(Second file) for 2337 loaded.
bollinger
     bol

Data(First file) for 3406 loaded.
Data(Second file) for 3406 loaded.
bollinger
     bollinger
2327    689427
2317    3765.1
2330  -1653.35
2337   1220.67
2492    186539
...        ...
2105       NaN
2349       NaN
2618       NaN
3596       NaN
0058       NaN

[100 rows x 1 columns]
Processing 3231
Data(First file) for 3231 loaded.
Data(Second file) for 3231 loaded.
bollinger
     bollinger
2327    689427
2317    3765.1
2330  -1653.35
2337   1220.67
2492    186539
...        ...
2105       NaN
2349       NaN
2618       NaN
3596       NaN
0058       NaN

[100 rows x 1 columns]
Processing 2882
Data(First file) for 2882 loaded.
Data(Second file) for 2882 loaded.
bollinger
     bollinger
2327    689427
2317    3765.1
2330  -1653.35
2337   1220.67
2492    186539
...        ...
2105       NaN
2349       NaN
2618       NaN
3596       NaN
0058       NaN

[100 rows x 1 columns]
Processing 2885
Data(First file) for 2885 loaded.
Data(Second file) for 2885 loaded.
bollinger
     bollinger
2327    6

KeyboardInterrupt: 